In [9]:
import pathlib as pl
import hashlib as hl
import pandas as pd

repo_path = pl.Path(".").resolve().parent

local_run = True

local_imbb_mount = pl.Path("/mounts/hilbert/project/projects/medbioinf")
remote_imbb_path = pl.Path("/gpfs/project/projects/medbioinf")

data_root_path = local_imbb_mount if local_run else remote_imbb_path

subfolders = [
    "data/00_RESTRUCTURE/project-centric/hgsvc/pacbio_hifi/",
]

###########

df = pd.read_csv(
    repo_path.joinpath("samples", "missing-fastq.tsv"),
    header=0, sep="\t"
)

print(df.loc[df["path_hash"] == "998085e158061f627df3ad492aa91438", "input_path"].values[0])

raise

###########




def merge_overlapping_paths(abs_path, rel_path):
    """Why the f*** does difflib not work on
    strings generated from paths???
    """
    top_level = abs_path
    connect_level = rel_path.split("/")[0]
    while 1:
        if top_level.name == connect_level:
            break
        top_level = top_level.parent
        if top_level == pl.Path("/"):
            raise ValueError("cannot merge")
    return top_level.parent
        
missing_fastqs = []
for subfolder in subfolders:
    search_path = data_root_path.joinpath(subfolder)
    for lst_file in search_path.glob("**/sample-folder.lst"):
        with open(lst_file, "r") as listing:
            for line in listing:
                top_level = merge_overlapping_paths(search_path, line)
                sample_folder = top_level.joinpath(line.strip())
                assert sample_folder.is_dir()
                if "20220831_JAX_HiFi" in str(sample_folder):
                    continue
                for bam_file in sample_folder.glob("**/*.bam"):
                    if "subreads" in bam_file.name:
                        continue
                    fastq = bam_file.with_suffix(".fastq.gz")
                    plain_fastq = bam_file.with_suffix(".fastq")
                    if not (fastq.is_file() and plain_fastq.is_file()):
                        rel_path = str(fastq.relative_to(sample_folder))
                        path_hash = hl.md5(rel_path.encode("utf-8")).hexdigest()
                        if local_run:
                            bam_path = str(bam_file).replace(str(local_imbb_mount), str(remote_imbb_path))
                            fastq_path = str(fastq).replace(str(local_imbb_mount), str(remote_imbb_path))
                        else:
                            bam_path = str(bam_file)
                            fastq_path = str(fastq)
                        missing_fastqs.append(
                            (path_hash, rel_path, bam_path, fastq_path)
                        )
                    else:
                        assert plain_fastq.is_file()
                        # someone uploaded uncompressed fastq ...
                        rel_path = str(fastq.relative_to(sample_folder))
                        path_hash = hl.md5(rel_path.encode("utf-8")).hexdigest()
                        if local_run:
                            input_path = str(plain_fastq).replace(str(local_imbb_mount), str(remote_imbb_path))
                            fastq_path = str(fastq).replace(str(local_imbb_mount), str(remote_imbb_path))
                        else:
                            input_path = str(plain_fastq)
                            fastq_path = str(fastq)
                        missing_fastqs.append(
                            (path_hash, rel_path, input_path, fastq_path)
                        )
                        

df = pd.DataFrame.from_records(
    missing_fastqs,
    columns=["path_hash", "rel_path", "input_path", "output_path"]
)

df.to_csv(
    repo_path.joinpath("samples", "missing-fastq.tsv"),
    header=True, index=False, sep="\t"
)

/gpfs/project/projects/medbioinf/data/00_RESTRUCTURE/project-centric/hgsvc/pacbio_hifi/20200731_CHS_PacBio_HG00514_HiFi_reseq/m54329U_200717_235548.ccs.bam


RuntimeError: No active exception to reraise